In [128]:
import os
import pandas as pd
import sqlite3

In [129]:
os.chdir('/Users/magdalena/OneDrive - Queen Mary, University of London/bezzlab/research/projects/phospho_logician/')
# os.chdir('/home/mhuebner/Desktop/bezzlab/research/projects/phospho_logician/')

In [130]:
# selecting cell line
cline = 'NTERA2'  # HL60, MCF7, NTERA2
cline_sql = 'NTERA-2 clone D1'  # HL-60, MCF-7, NTERA-2 clone D1

In [131]:
conn = sqlite3.connect('data/external/chemphopro.db')

In [132]:
# data frame with "where...." adding SQL Query parameters
df = pd.read_sql_query(f"SELECT * FROM Observation WHERE cell_line = '{cline_sql}' ;", conn)

# remove rows which have "-888.0000" values for fold change and p-value
df = df[df['p_value'] > -700]

# keep only S, T and Y as phosphosite residues (exclude X0 and M residues)
searchfor = ['\\(S', '\\(T', '\\(Y']
df = df[df['substrate'].str.contains('|'.join(searchfor))]

# duplicate entries (multiple log2fcs for the same perturbagen and phosphosite) are averaged out
df = df.groupby(['perturbagen', 'substrate'], as_index=False).mean()

# add the target protein (the protein that a phosphosite is on) to df
df['tprot'] = df['substrate'].str.replace(r'\(.*$', "")

# add the phosphosite residue
df.loc[df['substrate'].str.contains('\\(S'), 'residue'] = 's'
df.loc[df['substrate'].str.contains('\\(T'), 'residue'] = 't'
df.loc[df['substrate'].str.contains('\\(Y'), 'residue'] = 'y'

/var/folders/l3/vxqxpwf10_x9jp7g4_x9yy9m0000gn/T/ipykernel_70366/2919929.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df = df.groupby(['perturbagen', 'substrate'], as_index=False).mean()
/var/folders/l3/vxqxpwf10_x9jp7g4_x9yy9m0000gn/T/ipykernel_70366/2919929.py:15: FutureWarning: The default value of regex will change from True to False in a future version.
  df['tprot'] = df['substrate'].str.replace(r'\(.*$', "")


In [133]:
df

,perturbagen,substrate,fold_change,p_value,cv,tprot,residue
0,AC220,AAAS(S495),-0.065521,0.779794,0.146543,AAAS,s
1,AC220,AAGAB(S310),-0.275035,0.442984,0.293978,AAGAB,s
2,AC220,AAGAB(S311),-0.430552,0.237783,0.296208,AAGAB,s
3,AC220,AAK1(S14),-0.419871,0.285785,0.469502,AAK1,s
4,AC220,AAK1(S21),-0.014980,0.960226,0.300132,AAK1,s
...,...,...,...,...,...,...,...
981353,Vemurafenib,ZYX(T270),0.238530,0.404952,0.290054,ZYX,t
981354,Vemurafenib,ZYX(T274),-3.307531,0.079557,1.732051,ZYX,t
981355,Vemurafenib,ZZEF1(S1518),-0.546650,0.354334,0.606091,ZZEF1,s
981356,Vemurafenib,ZZZ3(S113),-0.116535,0.784741,0.438533,ZZZ3,s


In [134]:
# export df to csv
df.to_csv(f'data/processed/facts/observations_{cline}.csv', index=False)

In [135]:
# write Prolog file and assign down/up/unaffected based on p-value and log2fc
with open(f'models/facts/perturbs_{cline}.pl', 'w') as file:
    for index, row in df.iterrows():
        if row['p_value'] <= 0.05 and row['fold_change'] < 0:
            var1 = "perturbs('{}', '{}', '{}', {}, {}, {}, {}).".format(row['perturbagen'], row['substrate'],
                                                                        row['tprot'], row['residue'], 'down',
                                                                        row['fold_change'], row['p_value'])
        elif row['p_value'] <= 0.05 and row['fold_change'] > 0:
            var1 = "perturbs('{}', '{}', '{}', {}, {}, {}, {}).".format(row['perturbagen'], row['substrate'],
                                                                        row['tprot'], row['residue'], 'up',
                                                                        row['fold_change'], row['p_value'])
        else:
            var1 = "perturbs('{}', '{}', '{}', {}, {}, {}, {}).".format(row['perturbagen'], row['substrate'],
                                                                        row['tprot'], row['residue'], 'unaffected',
                                                                        row['fold_change'], row['p_value'])
        file.write(var1 + '\n')
